# Bert Sentiment Test

In [15]:
import sys
sys.path.append("../../../")

In [16]:
import numpy as np
import torch
from IPython.core.display import display, HTML
from transformers import BertTokenizer

from modules.lrp_bert_modules import LRPBertForSequenceClassification

from visualization.heatmap import html_heatmap

In [17]:
from torch import nn

Load model

In [3]:
# print("Loading model...")
# config_path = "bert-sst-config.pt"
# state_dict_path = "bert-sst.pt"

# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
# model = LRPBertForSequenceClassification(torch.load(config_path))
# model.load_state_dict(torch.load(state_dict_path))
# model.eval()
# print("Done.")

Loading model...
Done.


In [ ]:
# CIURRENTLY THIS IS WORKING WITH ORIGINAL MODEL FROM CLINICAL BERT

In [40]:
#load original model from clinicalBERT
# config_path = "./bert_config.json"
# model_path = "./pytorch_model.bin"
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


model_1layer_discharge_config_path = "../../../../model/newest_ClinicalBERTPretained_discharge_210421/config.json"
model_1layer_discharge_model_path = "../../../../model/newest_ClinicalBERTPretained_discharge_210421/pytorch_model.bin"

model_3layer_discharge_path = "../../../../model/newest_3layerClinicalBERTPretained_discharge_210421/"

#load 1 layer 
model1 = LRPBertForSequenceClassification.from_pretrained(model_1layer_discharge_path, num_labels = 1)
model2 = LRPBertForSequenceClassification(torch.load(model_1layer_discharge_config_path))
model2.load_state_dict(torch.load(model_1layer_discharge_model_path,map_location='cpu'))

model3 = LRPBertForSequenceClassification.from_pretrained(model_3layer_discharge_path, num_labels = 1)
# model = LRPBertForSequenceClassification.from_pretrained(torch.load(model_path))

# model = torch.load(model_path)
print(model1)


UnpicklingError: invalid load key, '{'.

In [30]:
print(model2)


Set up example

In [37]:
# test_example = "It's a lovely film with wonderful performances by Buy and " \
#                "Accorsi."

test_example = ' he, has, experienced, acute, on, chronic, diastolic, heart, failure, in the setting of volume overload due to his sepsis.'

In [38]:
test_example

' he, has, experienced, acute, on, chronic, diastolic, heart, failure, in the setting of volume overload due to his sepsis.'

Run normal forward pass

In [39]:
model.eval()
inputs = tokenizer(test_example, return_tensors="pt")
logits = model(**inputs).logits.squeeze()

m = nn.Sigmoid()
logits = torch.squeeze(m(logits)).detach().cpu().numpy()

print(logits)
# classes = ["<unk>", "positive", "negative", "neutral"]
classes = ["not_admitted", "admitted"]

# print("Logit Scores:")
# for c, score in zip(classes, logits):
#     print("{}: {}".format(c, score))

0.96026367


Run attribution forward pass

In [33]:
inputs = tokenizer(test_example, return_tensors="pt")
model.attr()
output = model(**inputs)

print("Attr Forward Pass Output:")
print(output)

Attr Forward Pass Output:
[[3.005818]]


Run LRP

In [34]:
tokens = tokenizer.tokenize(test_example)
rel_y = np.zeros(output.shape)
rel_y[:, 0] = output[:, 0]
rel_word, rel_pos, rel_type, rel_embed = model.attr_backward(rel_y, eps=.1)
rel_word = np.sum(rel_word[0, 1:-1], -1)
rel_pos = np.sum(rel_pos[0, 1:-1], -1)
rel_type = np.sum(rel_type[0, 1:-1], -1)
rel_embed = np.sum(rel_embed[0, 1:-1], -1)

print("LRP Scores:")
for t, s in zip(tokens, rel_embed):
    print(t, s, sep=": ")
    
print("Relevance of word embeddings:")
display(HTML(html_heatmap(tokens, list(rel_word))))

print("Relevance of positional embeddings:")
display(HTML(html_heatmap(tokens, list(rel_pos))))

print("Relevance of type embeddings:")
display(HTML(html_heatmap(tokens, list(rel_type))))

print("Relevance of combined embeddings:")
display(HTML(html_heatmap(tokens, list(rel_embed))))

LRP Scores:
he: 0.046435107968599586
has: 0.006323041011875523
experienced: -0.12908340886646283
acute: -0.36771864139854116
on: -0.11877602452014005
chronic: 0.18068827691213424
dia: 0.16323471084273566
##sto: -0.30414113223838124
##lic: 0.18059337185346486
heart: -0.13958726169712377
failure: 0.1647037468879956
in: -0.07315569959288457
the: 0.159346919019919
setting: 0.06668113958085745
of: -0.06719082451758973
volume: -0.08085323940011024
over: -0.06246023042943781
##load: -0.04087739198209185
due: -0.09033322024342459
to: -0.16731615607760178
his: 0.044149933816612585
sep: -0.706693439985873
##sis: -0.7826203551596715
.: 0.04353143368002793
Relevance of word embeddings:


Relevance of positional embeddings:


Relevance of type embeddings:


Relevance of combined embeddings:


In [55]:
def get_LRP_derived_token_scores():
    
    
    
    

IndexError: index 1 is out of bounds for axis 1 with size 1